In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/llm_finetuning/llm_finetuning/notebooks'
curr_filename = "vanilla_rte_baseline"
!pip install -q transformers accelerate bitsandbytes datasets

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/llm_finetuning/llm_finetuning/notebooks'
/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments, AdamW
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
import warnings
warnings.simplefilter("ignore")
import time

# Start the timer
start_time = time.time()

# Set seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Load RTE dataset
rte_dataset = load_dataset("glue", "rte")

# Load HANS dataset
hans_data = load_dataset("hans", split="validation")

# Define tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Preprocess RTE data
# def preprocess_rte(examples):
#     return tokenizer(
#         examples["sentence1"],
#         examples["sentence2"],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )

# # Preprocess HANS data
# def preprocess_hans(examples):
#     return tokenizer(
#         examples["premise"], examples["hypothesis"],  # Premise and hypothesis
#         truncation=True, padding="max_length", max_length=128
#     )

def manipulate_inputs_rte(batch):
    encoding = tokenizer(
        [f'Given the statement "{sentence1}", does it necessarily follow that "{sentence2}" is true?'
         for sentence1, sentence2 in zip(batch["sentence1"], batch["sentence2"])],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors='pt'
    )
    batch["input_ids"] = encoding["input_ids"].squeeze()
    batch["attention_mask"] = encoding["attention_mask"].squeeze()
    return batch

# Prepare the inputs with the fixed context for HANS
def manipulate_inputs_hans(batch):
    fixed_context = "Given the premise, does the hypothesis hold true? "
    encoding = tokenizer(
        [f'{fixed_context} Premise: {premise} Hypothesis: {hypothesis}'
         for premise, hypothesis in zip(batch["premise"], batch["hypothesis"])],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors='pt'
    )
    batch["input_ids"] = encoding["input_ids"].squeeze()
    batch["attention_mask"] = encoding["attention_mask"].squeeze()
    return batch

rte_dataset = rte_dataset.map(manipulate_inputs_rte, batched=True)
hans_dataset = hans_data.map(manipulate_inputs_hans, batched=True)

# Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Few-shot training
few_shot_sample_size = [2, 32, 128]
num_epochs = 40
batch_size = 32
learning_rate = 1e-5
num_runs = 10

results_df = pd.DataFrame(columns=["n", "run", "in_domain_accuracy", "out_of_domain_accuracy"])

for run_idx in range(num_runs):
  for n in few_shot_sample_size:

        # Reinitialize the model
        model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # Select n examples per class from RTE training data
        entailment_indices = np.where(np.array(rte_dataset["train"]["label"]) == 1)[0]
        non_entailment_indices = np.where(np.array(rte_dataset["train"]["label"]) == 0)[0]
        entailment_indices = np.random.choice(entailment_indices, n, replace=False)
        non_entailment_indices = np.random.choice(non_entailment_indices, n, replace=False)
        indices = np.concatenate([entailment_indices, non_entailment_indices])

        train_dataset = rte_dataset["train"].select(indices)

        # Training arguments
        training_args = TrainingArguments(
            output_dir="./results",
            overwrite_output_dir=True,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            learning_rate=learning_rate,
            weight_decay=0.0,
            save_steps=10_000,
            save_total_limit=2
        )

        # Trainer setup
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            compute_metrics=compute_metrics,
            optimizers=(optimizer, None),
        )

        # Train the model
        trainer.train()

        # Evaluate in-domain (RTE)
        print(f"Evaluating in-domain performance for n={n}...")
        in_domain_results = trainer.evaluate(eval_dataset=rte_dataset["validation"])
        in_domain_accuracy = in_domain_results["eval_accuracy"]

        # Evaluate out-of-domain (HANS)
        print(f"Evaluating out-of-domain performance for n={n}...")
        out_of_domain_results = trainer.evaluate(eval_dataset=hans_dataset)
        out_of_domain_accuracy = out_of_domain_results["eval_accuracy"]

        print(f"n={n}, run={run_idx}, In-domain Accuracy: {in_domain_accuracy}, Out-of-domain Accuracy: {out_of_domain_accuracy}")

        # Save results
        new_row = pd.DataFrame({
            "n": [n],
            "run": [run_idx],
            "in_domain_accuracy": [in_domain_accuracy],
            "out_of_domain_accuracy": [out_of_domain_accuracy],
            "in_domain_loss": [in_domain_results["eval_loss"]],
            "out_of_domain_loss": [out_of_domain_results["eval_loss"]],
            "in_domain_runtime": [in_domain_results["eval_runtime"]],
            "out_of_domain_runtime": [out_of_domain_results["eval_runtime"]],
            "in_domain_samples_per_second": [in_domain_results["eval_samples_per_second"]],
            "out_of_domain_samples_per_second": [out_of_domain_results["eval_samples_per_second"]],
            "in_domain_steps_per_second": [in_domain_results["eval_steps_per_second"]],
            "out_of_domain_steps_per_second": [out_of_domain_results["eval_steps_per_second"]],
        })

        print(new_row)
        results_df = pd.concat([results_df, new_row], ignore_index=True)


end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Total Runtime: {elapsed_time:.4f} seconds")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

...

Evaluating out-of-domain performance for n=32...
n=32, run=9, In-domain Accuracy: 0.5270758122743683, Out-of-domain Accuracy: 0.49883333333333335
    n  run  in_domain_accuracy  out_of_domain_accuracy  in_domain_loss  \
0  32    9            0.527076                0.498833        2.322063   

   out_of_domain_loss  in_domain_runtime  out_of_domain_runtime  \
0            0.766501             1.9582                207.213   

   in_domain_samples_per_second  out_of_domain_samples_per_second  \
0                       141.459                           144.779   

   in_domain_steps_per_second  out_of_domain_steps_per_second  
0                      17.874                          18.097  


Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Evaluating in-domain performance for n=128...


Evaluating out-of-domain performance for n=128...
n=128, run=9, In-domain Accuracy: 0.5992779783393501, Out-of-domain Accuracy: 0.5
     n  run  in_domain_accuracy  out_of_domain_accuracy  in_domain_loss  \
0  128    9            0.599278                     0.5        3.266377   

   out_of_domain_loss  in_domain_runtime  out_of_domain_runtime  \
0            1.028545             1.9293               207.1046   

   in_domain_samples_per_second  out_of_domain_samples_per_second  \
0                       143.572                           144.854   

   in_domain_steps_per_second  out_of_domain_steps_per_second  
0                      18.141                          18.107  
Total Runtime: 9410.7152 seconds


In [3]:
print(results_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>n</th>
      <th>run</th>
      <th>in_domain_accuracy</th>
      <th>out_of_domain_accuracy</th>
      <th>in_domain_loss</th>
      <th>out_of_domain_loss</th>
      <th>in_domain_runtime</th>
      <th>out_of_domain_runtime</th>
      <th>in_domain_samples_per_second</th>
      <th>out_of_domain_samples_per_second</th>
      <th>in_domain_steps_per_second</th>
      <th>out_of_domain_steps_per_second</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2</td>
      <td>0</td>
      <td>0.548736</td>
      <td>0.494133</td>
      <td>0.967875</td>
      <td>0.706550</td>
      <td>1.8819</td>
      <td>204.1095</td>
      <td>147.192</td>
      <td>146.980</td>
      <td>18.598</td>
      <td>18.372</td>
    </tr>
    <tr>
      <th>1</th>
      <td>32</td>
      <td>0</td>
      <td>0.527076</td>
      <td>0.498833</td>
      <td>2.322063</td>
      <td>0.7

In [ ]:
curr_filename = 'valina_rte_baseline'
%cd '/content/drive/MyDrive/llm_finetuning/llm_finetuning/notebooks'
# Save results to a CSV file
results_df.to_csv(curr_filename + ".csv", index=False)